In [28]:
import pandas as pd
import mysql.connector
import numpy as np

mydb=mysql.connector.connect(
     host="127.0.0.1",
     port="3306",
     user="root",
     password="2389",
     database="bikestores"
)
cur=mydb.cursor()

try:
    cur.execute("SELECT employeeNumber,lastName,firstName,jobTitle FROM employees")
    results=cur.fetchall()
    df_employees=pd.DataFrame(results,columns=[i[0] for i in cur.description])

    cur.execute("select customerNumber, city, country, salesRepEmployeeNumber from customers")
    results=cur.fetchall()
    df_customers=pd.DataFrame(results,columns=[i[0] for i in cur.description]) 

    cur.execute("select customerNumber, amount from payments")
    results=cur.fetchall()
    df_payments=pd.DataFrame(results,columns=[i[0] for i in cur.description])
    
    cur.execute("select orderNumber,customerNumber,orderDate from orders")
    results=cur.fetchall()
    df_orders=pd.DataFrame(results,columns=[i[0] for i in cur.description])
    
    cur.execute("select productCode,productName,productLine from products")
    results=cur.fetchall()
    df_products=pd.DataFrame(results,columns=[i[0] for i in cur.description])

    cur.execute("select productLine from productlines")
    results=cur.fetchall()
    df_productsline=pd.DataFrame(results,columns=[i[0] for i in cur.description])

    cur.execute("select orderNumber,productCode,quantityOrdered,priceEach from orderdetails")
    results=cur.fetchall()
    df_orderdetails=pd.DataFrame(results,columns=[i[0] for i in cur.description])

  
except:
    pass

In [ ]:
df_orderdetails

In [ ]:
df_customers.fillna(0,inplace=True)

df_customers['salesRepEmployeeNumber'] = df_customers['salesRepEmployeeNumber'].astype(int)


mydb1=mysql.connector.connect(
    host="127.0.0.1",
    port="3306",
    user="root",
    password="2389",
    database="staging"
)
cur_staging=mydb1.cursor()

def insert_into_staging(cur,table_name,data_frame): 
      
     values=[tuple(x) for x in data_frame.values]
     query="INSERT INTO "+table_name+" VALUES "
     placeholders=', '.join(['%s']*len(data_frame.columns))
     query_all=query+ f"({placeholders})" 
     print(query_all)
     try:
         cur.executemany(query_all,values)
         mydb1.commit()
     except:
         mydb1.rollback()

try: 
    cur_staging.execute("create table if not exists employees ( employee_id int primary key, "
                        + "last_name varchar(100), "
                        +"first_name varchar(100), "
                        +"job_title varchar(100) )"                    
                        )
    cur_staging.execute("create table if not exists customers ( customer_id int primary key, "
                        + "city varchar(100), "
                        +"country varchar(100), "
                        +" sales_rep int)"          
                        )
    cur_staging.execute("create table if not exists payments ( customer_id int, "
                        + "amount DECIMAL(10,2) )"   
                        )
    # Tạo bảng "orders"
    cur_staging.execute("""
        CREATE TABLE IF NOT EXISTS orders (
            order_id INT PRIMARY KEY,
            customer_id INT,
            order_date DATE
        )
    """)
    # Tạo bảng "products"
    cur_staging.execute("""
        CREATE TABLE IF NOT EXISTS products (
            product_code VARCHAR(15) PRIMARY KEY,
            product_name VARCHAR(100),
            product_line VARCHAR(50)
        )
    """)
    # Tạo bảng "productlines"
    cur_staging.execute("""
        CREATE TABLE IF NOT EXISTS productlines (
            product_line VARCHAR(50) PRIMARY KEY
        )
    """)
    # Tạo bảng "orderdetails"
    # Tạo bảng "orderdetails" mà không tham chiếu khóa ngoại
 
    cur_staging.execute("""
        CREATE TABLE IF NOT EXISTS orderdetails (
            order_id INT,
            product_code VARCHAR(15),
            quantity_ordered INT,
            price_each DECIMAL(10,2)
        )
    """)

except Exception as e:
    print(f"Error: {e}")
    mydb1.rollback()


insert_into_staging(cur_staging,"employees",df_employees)
insert_into_staging(cur_staging,"customers",df_customers)
insert_into_staging(cur_staging,"payments",df_payments)
insert_into_staging(cur_staging,"products",df_products)
insert_into_staging(cur_staging,"productlines",df_productsline)
insert_into_staging(cur_staging,"orderdetails",df_orderdetails)
